In [23]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from scipy.spatial import cKDTree
import open3d as o3d
from collections import defaultdict

import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from pc_seg.pc_label_map import color_map_dict

In [24]:
# Invert the colormap to get RGB → label index
rgb_to_label = {tuple(v[0]): k for k, v in color_map_dict.items()}
label_to_name = {k: v[1] for k, v in color_map_dict.items()}

def get_label_from_color(color_array):
    """Match color to closest known semantic color label."""
    labels = []
    for c in color_array:
        # Round to 3 decimals to avoid float precision mismatch
        key = tuple(np.round(c, 3))
        label = rgb_to_label.get(key, -1)  # -1 if unmatched
        labels.append(label)
    return np.array(labels)

In [25]:
# Load GT and predicted point clouds
gt_pcd = o3d.io.read_point_cloud("../docs/SmartLab_2024_E57_Single_5mm_SEG_colors.ply")   # full resolution
pred_pcd = o3d.io.read_point_cloud("../docs/Smartlab_s3dis_label_pointnet2_x6_0.03_20250422.ply")   # downsampled

In [11]:
o3d.visualization.draw_geometries([gt_pcd], point_show_normal=False)

In [12]:
o3d.visualization.draw_geometries([pred_pcd], point_show_normal=False)

In [26]:
def print_label_counts(label_array, name="Point Cloud"):
    print(f"\n📦 Label counts for {name}:")
    unique_labels, counts = np.unique(label_array, return_counts=True)
    for label, count in zip(unique_labels, counts):
        class_name = label_to_name.get(label, f"Class {label}")
        print(f"  {label:2d} ({class_name:10s}): {count} points")

In [27]:
# Move the point cloud to its min(x,y,z) corner
 
def move_to_corner(points):    
    # Find the minimum x, y, z
    min_xyz = points.min(axis=0)
    # Translate the point cloud so that the min corner becomes the origin
    moved_points = points - min_xyz
    
    return moved_points

moved_points = move_to_corner(np.array(gt_pcd.points))
gt_pcd.points = o3d.utility.Vector3dVector(moved_points)

In [29]:
# Extract point coordinates and colors
gt_points = np.asarray(gt_pcd.points)
gt_origin_colors = np.asarray(gt_pcd.colors)
gt_colors = np.round(gt_origin_colors, 1)

pred_points = np.asarray(pred_pcd.points)
pred_origin_colors = np.asarray(pred_pcd.colors)
pred_colors = np.round(pred_origin_colors, 1)

# Convert color to semantic labels
gt_labels = get_label_from_color(gt_colors)
pred_labels = get_label_from_color(pred_colors)

In [30]:
# Ground Truth
print_label_counts(gt_labels, name="Ground Truth")

# Prediction
print_label_counts(pred_labels, name="Prediction")


📦 Label counts for Ground Truth:
   0 (ceiling   ): 5701035 points
   1 (floor     ): 5167644 points
   2 (wall      ): 11348550 points
   4 (column    ): 173882 points
   5 (window    ): 549543 points
   6 (door      ): 1451390 points
   7 (table     ): 243330 points
   8 (chair     ): 99590 points
   9 (sofa      ): 190263 points
  11 (board     ): 488769 points
  12 (clutter   ): 10552776 points

📦 Label counts for Prediction:
   0 (ceiling   ): 130218 points
   1 (floor     ): 120958 points
   2 (wall      ): 277981 points
   4 (column    ): 5052 points
   5 (window    ): 208 points
   6 (door      ): 20088 points
   7 (table     ): 114 points
   8 (chair     ): 7 points
  10 (bookcase  ): 63449 points
  11 (board     ): 14 points
  12 (clutter   ): 248811 points


In [31]:
# Filter out unmatched labels (e.g. -1)
valid_mask = (gt_labels != -1)
gt_points = gt_points[valid_mask]
gt_labels = gt_labels[valid_mask]

# KDTree nearest neighbor matching
tree = cKDTree(pred_points)
_, indices = tree.query(gt_points, k=1)
matched_pred_labels = pred_labels[indices]

# Overall accuracy
overall_accuracy = accuracy_score(gt_labels, matched_pred_labels)
print(f"\n✅ Overall Accuracy: {overall_accuracy:.3f}")

# Per-class accuracy
print("\n📋 Per-Class Accuracy:")
class_counts = defaultdict(int)
correct_counts = defaultdict(int)

for true, pred in zip(gt_labels, matched_pred_labels):
    class_counts[true] += 1
    if true == pred:
        correct_counts[true] += 1

for label in sorted(class_counts.keys()):
    correct = correct_counts[label]
    total = class_counts[label]
    acc = correct / total if total > 0 else 0.0
    class_name = label_to_name.get(label, f"Class {label}")
    print(f"  {label:2d} ({class_name:10s}): {acc:.3f} ({correct}/{total})")


✅ Overall Accuracy: 0.645

📋 Per-Class Accuracy:
   0 (ceiling   ): 0.839 (4783406/5701035)
   1 (floor     ): 0.947 (4895896/5167644)
   2 (wall      ): 0.642 (7283663/11348550)
   4 (column    ): 0.000 (0/173882)
   5 (window    ): 0.002 (1202/549543)
   6 (door      ): 0.110 (158963/1451390)
   7 (table     ): 0.009 (2088/243330)
   8 (chair     ): 0.000 (0/99590)
   9 (sofa      ): 0.000 (0/190263)
  11 (board     ): 0.000 (0/488769)
  12 (clutter   ): 0.575 (6064681/10552776)


In [32]:
# Optional: classification report
# Ensure all labels used in GT or prediction are covered
all_labels = sorted(set(gt_labels) | set(matched_pred_labels))

print(classification_report(
    gt_labels,
    matched_pred_labels,
    labels=all_labels,
    target_names=[label_to_name.get(i, f"Class {i}") for i in all_labels]
))

C:\Users\yanpe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\yanpe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\yanpe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_classification.py:15

              precision    recall  f1-score   support

     ceiling       0.95      0.84      0.89   5701035
       floor       0.97      0.95      0.96   5167644
        wall       0.62      0.64      0.63  11348550
      column       0.00      0.00      0.00    173882
      window       0.28      0.00      0.00    549543
        door       0.14      0.11      0.12   1451390
       table       0.38      0.01      0.02    243330
       chair       0.00      0.00      0.00     99590
        sofa       0.00      0.00      0.00    190263
    bookcase       0.00      0.00      0.00         0
       board       0.00      0.00      0.00    488769
     clutter       0.58      0.57      0.58  10552776

    accuracy                           0.64  35966772
   macro avg       0.33      0.26      0.27  35966772
weighted avg       0.67      0.64      0.65  35966772



C:\Users\yanpe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\yanpe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [14]:
# Compute the centroid
gt_centroid = gt_points.mean(axis=0)
pred_centroid = pred_points.mean(axis=0)

print(gt_centroid)
print(pred_centroid)

[4.19205701 1.61344356 0.22196501]
[5.20587027 5.64833263 1.76726065]


In [15]:
gt_min_xyz = gt_points.min(axis=0)
pred_min_xyz = pred_points.min(axis=0)

print(gt_min_xyz)
print(pred_min_xyz)

[-1.03750002 -4.06758022 -1.56200004]
[0.         0.00400019 0.00703452]


In [6]:


# Extract numpy arrays
gt_points = np.asarray(gt_pcd.points)
gt_colors = np.asarray(gt_pcd.colors)  # RGB in [0.0, 1.0]

pred_points = np.asarray(pred_pcd.points)
pred_colors = np.asarray(pred_pcd.colors)

# Map RGB colors to semantic class labels
gt_labels = get_label_from_color(gt_colors)
pred_labels = get_label_from_color(pred_colors)

# Filter out unmatched labels (optional)
valid_mask = (gt_labels != -1)
gt_points = gt_points[valid_mask]
gt_labels = gt_labels[valid_mask]

# KDTree to find nearest predicted point for each GT point
tree = cKDTree(pred_points)
_, indices = tree.query(gt_points, k=1)

# Predicted labels by nearest point
matched_pred_labels = pred_labels[indices]

# Evaluate
print("Accuracy:", accuracy_score(gt_labels, matched_pred_labels))
print("Confusion Matrix:\n", confusion_matrix(gt_labels, matched_pred_labels))
print("Classification Report:\n", classification_report(gt_labels, matched_pred_labels))


Accuracy: 0.25423009826106197
Confusion Matrix:
 [[      0       0       0       0       0       0       0       0]
 [1833676       0   10518 1992383      34 1645166       0  219258]
 [  91965       0 4283931  711422       0    2188       0   78138]
 [1227032       0 4877474 3870461       0  742960       0  630623]
 [      0       0       0       0       0       0       0       0]
 [  18970       0   59926     541       0   94445       0       0]
 [  31691       0  166651  182922       0  163731       0    4548]
 [1246009       0 3884928 3739601       0 1416037       0  266201]]


C:\Users\yanpe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\yanpe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\yanpe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_classification.py:15

Classification Report:
               precision    recall  f1-score   support

          -1       0.00      0.00      0.00         0
           0       0.00      0.00      0.00   5701035
           1       0.32      0.83      0.46   5167644
           2       0.37      0.34      0.35  11348550
           3       0.00      0.00      0.00         0
           4       0.02      0.54      0.04    173882
           5       0.00      0.00      0.00    549543
          12       0.22      0.03      0.05  10552776

    accuracy                           0.25  33493430
   macro avg       0.12      0.22      0.11  33493430
weighted avg       0.24      0.25      0.21  33493430



C:\Users\yanpe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\yanpe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
